In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import numpy as np

In [3]:
device = torch.device("cpu")
print(device)

cpu


In [6]:
class EffNet(nn.Module):
    def __init__(self, n_classes):
        super(EffNet, self).__init__()
        self.b4 = EfficientNet.from_pretrained('efficientnet-b0')
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(1000, n_classes)

    def forward(self, image):
        x = self.b4(image)
        x = self.drop(x)
        out = self.fc(x)
        return out

net = EffNet(n_classes=2).to(device)
net.load_state_dict(torch.load('/home/rafael/Downloads/binary-classification-galaxies-1/utils/models/modelo_galaxias.pth', map_location=torch.device('cpu')))

net.eval()

Loaded pretrained weights for efficientnet-b0


EffNet(
  (b4): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
        

In [27]:
IMAGE_SHAPE = (200, 200)
classes = ['eliptical', 'spiral']

def predict_image(image_path, model, device, transform=None):

    if transform is None:
        transform = transforms.Compose([
            transforms.Resize(IMAGE_SHAPE), 
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=10),  
            transforms.ToTensor(),
        ])

    image = Image.open(image_path).convert('RGB')  # Certifique-se que a imagem está em RGB
    image = transform(image) 
    image = image.unsqueeze(0).to(device)  # Adiciona uma dimensão de lote

    model.eval()  # Define o modelo para modo de avaliação
    with torch.no_grad():
        output = model(image)
        probabilities = torch.softmax(output, dim=1)
        predicted_class_index = torch.argmax(probabilities, dim=1).item()
        confidence = torch.max(probabilities).item()

    predicted_class = classes[predicted_class_index]

    return predicted_class, confidence

In [29]:
# Exemplo de uso
image_path = '/home/rafael/Downloads/binary-classification-galaxies/data/filtered_data/eliptical/100322.jpg'
predicted_class, confidence = predict_image(image_path, net, device)

print(f"Classe Predita: {predicted_class}, Confiança: {confidence:.2f}")

Classe Predita: eliptical, Confiança: 1.00


In [30]:
# Exemplo de uso
image_path = '/home/rafael/Downloads/binary-classification-galaxies/data/filtered_data/spiral/100673.jpg'
predicted_class, confidence = predict_image(image_path, net, device)

print(f"Classe Predita: {predicted_class}, Confiança: {confidence:.2f}")

Classe Predita: spiral, Confiança: 1.00
